In [2]:
#using NBInclude
#@nbinclude(joinpath("Hofstadter","Hofstadter Single Particle in Theta Space.ipynb"))
#@nbinclude("../Single Particle/Lattice_Basis_Neighbors_Hops.ipynb")

In [3]:
using LinearAlgebra

In [16]:
function calc_states_hofstadter(Tx, Ty, dimH, sites,neighbors_list,
                                site_number,site_coord,
                                nb,LL,param_dict,pbc)
    N1 = length(Tx)
    N2 = length(Ty)
    EEA=zeros(N1, N2, dimH)
    UUA=zeros(Complex{Float64}, N1, N2, dimH, dimH)
    for it1 in 1:N1
        for it2 in 1:N2
            param_dict[θθ]=[ Tx[it1], Ty[it2] ]
            EEA[it1,it2,:], UUA[it1,it2,:,:] = eigen(get_Hofstadter_hamiltonian(sites,neighbors_list,
                                               site_number,site_coord,
                                               nb,LL,param_dict,pbc))
        end
    end
    return EEA, UUA
end

calc_states_hofstadter (generic function with 1 method)

In [ ]:
function calc_states_KM(Nx, Ny, N1, N2, p, q, Tx, Ty, dimH)
    N1 = length(Tx)
    N2 = length(Ty)
    EEA=zeros(Complex{Float64}, N1, N2, dimH)
    UUA=zeros(Complex{Float64}, N1, N2, dimH, dimH)
    for it1 in 1:N1
        for it2 in 1:N2
            EEA[it1,it2,:], UUA[it1,it2,:,:] = eigen( KM_Hamiltonian(Nx, Ny, t, p, q, Tx[it1], Ty[it2]) )
        end
    end
    return EEA, UUA
end

In [2]:
function calc_link_var(ψ1, ψ2)
    link_var = det(ψ1'*ψ2)
    return link_var/abs(link_var)
end

calc_link_var (generic function with 1 method)

In [4]:
function calc_link_vars(UUA)
    N1,N2,__ = size(UUA)
    UU=zeros(Complex{Float64}, N1, N2, 2)
    dirs = [ [1, 0], [0,1] ]
    for it1 in 1:N1
        for it2 in 1:N2
            for (idir, vdir) in enumerate(dirs)
                UU[it1,it2,idir] = calc_link_var(UUA[it1,it2,:,:],
                                                 UUA[mod(it1-1+vdir[1],N1)+1,mod(it2-1+vdir[2],N2)+1,:,:])
            end
        end
    end
    return UU 
end

calc_link_vars (generic function with 1 method)

In [5]:
function calc_F12(UU)
    F12 = log(  UU[:,:,1]*
                circshift(UU[:,:,2],(-1,0))/
               (circshift(UU[:,:,1],(0,-1))*
                UU[:,:,2]) 
            )
    return F12
end

calc_F12 (generic function with 1 method)